# Example: Prospect Theory, Basket Option

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
plt.style.use('bmh')
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import scipy
import seaborn
from tqdm import tqdm
import yfinance as yf
from scipy.stats import norm
from optimization_functions import *
from functions_multidim import *
from functions_multidim_adaptive import *
%load_ext autoreload
%autoreload 2
%aimport optimization_functions
%aimport functions_multidim   
%aimport functions_multidim_adaptive

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Download Stock Prices and create Training Periods

In [2]:
stocks = ["GOOGL","EBAY","AMZN","MSFT","AAPL"]
d = len(stocks)
Returns_train_tf_dict = {}

In [3]:
for s in stocks:
    S_train = yf.download(s, start="2010-01-02", end="2020-02-01")
    S_train = S_train.dropna()
    S_train = S_train.reset_index()
    S_train = S_train[["Date","Close"]]
    
    Returns_train = S_train["Close"].diff() / S_train["Close"].shift(1)
    Returns_train = Returns_train.dropna()
    # A Tensorflow Adaption
    Returns_train_tf_dict[s] = tf.reshape(Returns_train,(1,len(Returns_train)))
    
# Create a d dimensional vector
Returns_train_tf = tf.concat([Returns_train_tf_dict[s] for s in stocks],0)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Test Period 1: Feb 2020 - May 2020

# Description of the Algorithm

For $t = T-1, \cdots , 1$ :


For $j = 1,\dots, N_{iterations}$:

Sample $(\omega_1,\dots,\omega_t)=\omega^t$

Sample $(a_0,\dots,a_{t-1})=a^t$

For $k = 1,\dots, N_{measures}$:

Sample next states
$w^{t+1, (k),(i)} \sim \mathbb{P}_k \in \mathcal{P}_t(\omega^t)$ for $i= 1,\dots,N_{MC}$

<!-- Compute $\tilde{J}_t(\omega,a^t, \tilde{a}) = \min_k \frac{1}{N}\sum_{i=1} \tilde{\Psi}_{t+1}((\omega^t,a^t, \tilde{a}))$
Maximize w.r.t. $\widetilde{a}$:

$\widetilde{a}_t(\omega^t,a^t)=argmax_{\widetilde{a}} \tilde{J}_t(\omega,a^t, \tilde{a})$

Approximate

$\tilde{\Psi}_t(\omega^t, a^t):= \tilde{J}_t(\omega,a^t, \widetilde{a}_t(\omega^t,a^t))$ -->

The optimization step is:

1.)

Maximize
$$
\sup_{\widetilde{a} \in A}\inf_{ P \in \mathcal{P}_t(\omega^t)} E_P \left[\Psi_{t+1}\left(\omega^t \otimes_t \cdot, (a^{t},\widetilde{a})\right)\right]= \Psi_t(\omega^t,a^t)
$$
 w.r.t. parameters of $\tilde{a}(\omega^t,a^t)$:

 Alternative:

 By using a dual approach we have in the special case of a Wasserstein distance
 $$
\Psi_t(\omega^t,a^t)=\sup_{\widetilde{a} \in A}\inf_{ P \in \mathcal{P}_t(\omega^t)} E_P \left[\Psi_{t+1}\left(\omega^t \otimes_t \cdot, (a^{t},\widetilde{a})\right)\right]
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0} E_{\widehat{P}} \left[\inf_{z} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z, (a^{t},\widetilde{a})\right)+\lambda \|\cdot -z\|\right\}\right] - \lambda \varepsilon
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0} \frac{1}{N_{MC}} \sum_{i=1}^{N_{MC}} \left[\min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|x_i -z_j\|\right\}\right] - \lambda \varepsilon
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0}  \sum_{i=t}^{N-1} \left[\pi_i(\omega^t) \min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|\mathcal{R}_{i+1} -z_j\|\right\}\right] - \lambda \varepsilon
$$
2.)


Minimize
$$
\sum_{b=1}^B \left(\tilde{\Psi}_t(\omega^t_b, a^t_b)- \left(\sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0}  \sum_{i=t}^{N-1} \left[\pi_i(\omega^t) \min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|\mathcal{R}_{i+1} -z_j\|\right\}\right] - \lambda \varepsilon \right)  \right)^2
$$
 w.r.t. parameters of $\tilde{\Psi}_t$:

Finally, we set:
$$
\Omega^t \ni \omega^t =(\omega_1,\dots,\omega_{t}) \mapsto a_t^*(\omega^t):= \widetilde{a}_t^*\left(\omega^t,~\left(a_0^*,\dots,a_{t-1}^*(\omega_1,\dots,\omega_{t-1}\right)\right) \in A.
$$

# Training

Number of periods

In [5]:
T = 10

Initialize the dictionaries

Compute $\varepsilon_t$

In [7]:
C = np.max(np.abs(Returns_train_tf))
d = 5
N = Returns_train_tf.shape[1]
eps_list = []
for t in range(T):    
    astar = 2*C*np.sqrt(d)/((N+t)**(1/(2*np.ceil(d/2)))-1)
    term0 = C*np.sqrt(d)/2-astar
    term1 = np.log(C*np.sqrt(d)/(2*astar))*2*C*np.sqrt(d)*np.ceil(d/2)
    term2 = np.sum([math.comb(int(np.ceil(d/2)), k)*(2*C*np.sqrt(d))**k *(((C*np.sqrt(d)/2)**(1-k)-astar**(1-k))/(1-k)) for k in range(2, int(np.ceil(d/2)+1))])
    eps_list.append((64/2.7)*(astar+(N+t)**(-1/2)*(term0+term1+term2)))
eps_list

[3.1132966309336343,
 3.1139444567185115,
 3.1145917859965717,
 3.115238619314474,
 3.1158849572180713,
 3.116530800252412,
 3.117176148961739,
 3.1178210038894987,
 3.118465365578335,
 3.1191092345700926]

Set the parameters

In [8]:
inner_psi = 2000
inner_a = 500
tolerance_psi = 0.0001
tolerance_a = 0.00001
learning_rate_Psi = 0.001
learning_rate_a = 0.001
learning_rate_LAM = 0.001
# Number of Monte Carlo Simulations
N_MC = 128
N_MC_inner = 128
#Batch Sizes
Batch_size_a = 128
Batch_size_psi = 128

Train

In [23]:
#compute
print("Start optimization")
a_tilde_5dim_adaptive, _ = train_networks_ndim_adaptive(Returns_train_tf,T=T,inner_psi = inner_psi,
                   inner_a =inner_a, N_MC=N_MC, N_MC_inner=N_MC_inner, epsilon = eps_list, tolerance_psi =tolerance_psi,
                   tolerance_a = tolerance_a, learning_rate_Psi = learning_rate_Psi,learning_rate_a = learning_rate_a,
                                    learning_rate_LAM = learning_rate_LAM, Batch_size_a = Batch_size_a,
                                        Batch_size_psi = Batch_size_psi,print_intermediate_results = True) 
#save
file_save(a_tilde_5dim_adaptive ,"Basket","Robust_5dim_adaptive")

Start optimization
Start Backwards Iterations
#########
# t = 9 #
#########

a: -4.557800769805908
a: -4.48776388168335
a: -4.4077582359313965
a: -4.329706192016602
a: -4.293130397796631
a: -4.198901653289795
a: -4.084414482116699
a: -4.031955718994141
a: -3.988497018814087
a: -3.8968186378479004
a: -3.7981984615325928
a: -3.778900146484375
a: -3.6873438358306885
a: -3.6013059616088867
a: -3.55625057220459
a: -3.4088475704193115
a: -3.33833646774292
a: -3.306537389755249
a: -3.213087558746338
Psi: 3.1129696369171143
Psi: 0.9482808113098145
Psi: 0.49646082520484924
Psi: 0.380473256111145
Psi: 0.3049298822879791
Psi: 0.2535107135772705
Psi: 0.18547455966472626
Psi: 0.10374186933040619
Psi: 0.06760112196207047
Psi: 0.04867405444383621
Psi: 0.039820894598960876
Psi: 0.0368540920317173
Psi: 0.030847745016217232
Psi: 0.028519218787550926
Psi: 0.025602443143725395
Psi: 0.023273548111319542
Psi: 0.02219275012612343
Psi: 0.019330622628331184
Psi: 0.017967965453863144
Psi: 0.018116772174835205
P